Reference: https://github.com/enakai00/jupyter_tfbook

In [16]:
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180804)
tf.set_random_seed(20180804)

In [17]:
mnist = input_data.read_data_sets('./data/', one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [32]:
class SingleLayerNetwork:

    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()
    
    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name=('input'))
        
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]), name='weights')
            b1 = tf.Variable(tf.zeros([num_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name='hidden1')
        
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name='weights')
            b0 = tf.Variable(tf.zeros([10]), name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')
        
        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)
        
        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.histogram('weights_hidden', w1)
        tf.summary.histogram('biases\hidden', b1)
        tf.summary.histogram('weights_output', w0)
        tf.summary.histogram('biases_output', b0)
        
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
    
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.initialize_all_variables())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter('./tmp/mnist_sl_logs', sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

In [33]:
nn = SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x: batch_xs, nn.t: batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run(
            [nn.summary, nn.loss, nn.accuracy],
            feed_dict={nn.x: mnist.test.images, nn.t: mnist.test.labels})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)

C:\Users\shiny\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Step: 100, Loss: 2772.4091796875, Accuracy: 0.9144999980926514
Step: 200, Loss: 2097.42919921875, Accuracy: 0.9368000030517578
Step: 300, Loss: 1750.94970703125, Accuracy: 0.9487000107765198
Step: 400, Loss: 1954.9786376953125, Accuracy: 0.9366999864578247
Step: 500, Loss: 1473.0185546875, Accuracy: 0.9526000022888184
Step: 600, Loss: 1468.2294921875, Accuracy: 0.9549999833106995
Step: 700, Loss: 1156.179931640625, Accuracy: 0.9635999798774719
Step: 800, Loss: 1354.279296875, Accuracy: 0.9580000042915344
Step: 900, Loss: 1261.0205078125, Accuracy: 0.9621000289916992
Step: 1000, Loss: 1113.9140625, Accuracy: 0.967199981212616
Step: 1100, Loss: 1425.8262939453125, Accuracy: 0.9549999833106995
Step: 1200, Loss: 1205.927001953125, Accuracy: 0.9646000266075134
Step: 1300, Loss: 1000.9241333007812, Accuracy: 0.970300018787384
Step: 1400, Loss: 1112.850830078125, Accuracy: 0.96670001745224
Step: 1500, Loss: 1133.46630859375, Accuracy: 0.9660999774932861
Step: 1600, Loss: 971.9007568359375, Ac